In [ ]:
"""
2.156 Part Validation Dataset Builder
Complete pipeline from CAD models to augmented training images
"""

import os
import sys
import json
import time
import math
import shutil
from pathlib import Path
from datetime import datetime
import numpy as np
import cv2
from PIL import Image, ImageEnhance

# ============================================================================
# PART 1: DATASET ORGANIZATION & TRACKING
# ============================================================================

class PartDatasetManager:
    """Manage and organize your part collection"""
    
    def __init__(self, project_root="part_validation_project"):
        self.root = Path(project_root)
        self.root.mkdir(exist_ok=True)
        
        # Create directory structure
        self.cad_dir = self.root / "cad_models"
        self.rendered_dir = self.root / "rendered_images"
        self.augmented_dir = self.root / "augmented_images"
        self.logs_dir = self.root / "logs"
        
        for directory in [self.cad_dir, self.rendered_dir, 
                         self.augmented_dir, self.logs_dir]:
            directory.mkdir(exist_ok=True)
        
        self.manifest_file = self.root / "dataset_manifest.json"
        self.load_manifest()
    
    def load_manifest(self):
        """Load or create dataset manifest"""
        if self.manifest_file.exists():
            with open(self.manifest_file, 'r') as f:
                self.manifest = json.load(f)
        else:
            self.manifest = {
                "created": datetime.now().isoformat(),
                "parts": {},
                "categories": {},
                "statistics": {
                    "total_cad_models": 0,
                    "total_rendered_images": 0,
                    "total_augmented_images": 0
                }
            }
    
    def save_manifest(self):
        """Save manifest to file"""
        with open(self.manifest_file, 'w') as f:
            json.dump(self.manifest, f, indent=2)
    
    def add_part(self, part_name, category, source, file_path):
        """Register a new part in the dataset"""
        part_id = f"{category}_{part_name}"
        
        self.manifest["parts"][part_id] = {
            "name": part_name,
            "category": category,
            "source": source,
            "file_path": str(file_path),
            "added_date": datetime.now().isoformat(),
            "rendered": False,
            "augmented": False
        }
        
        if category not in self.manifest["categories"]:
            self.manifest["categories"][category] = []
        self.manifest["categories"][category].append(part_id)
        
        self.manifest["statistics"]["total_cad_models"] += 1
        self.save_manifest()
        
        print(f"✓ Added part: {part_name} ({category})")
    
    def organize_cad_files(self, source_dir):
        """Organize CAD files into category folders"""
        print("\n=== Organizing CAD Files ===")
        
        source_path = Path(source_dir)
        if not source_path.exists():
            print(f"✗ Source directory not found: {source_dir}")
            return
        
        cad_extensions = ['.step', '.stp', '.stl', '.iges', '.igs', '.obj']
        files = []
        
        for ext in cad_extensions:
            files.extend(source_path.glob(f"*{ext}"))
        
        print(f"Found {len(files)} CAD files")
        
        for file_path in files:
            print(f"\nFile: {file_path.name}")
            category = input("  Category (bolt/washer/nut/bracket/gear/bearing/other): ").strip().lower()
            
            if not category:
                category = "uncategorized"
            
            # Create category folder
            category_dir = self.cad_dir / category
            category_dir.mkdir(exist_ok=True)
            
            # Copy file
            dest_path = category_dir / file_path.name
            shutil.copy2(file_path, dest_path)
            
            # Add to manifest
            part_name = file_path.stem
            self.add_part(part_name, category, "manual_download", dest_path)
        
        print(f"\n✓ Organized {len(files)} files")
        self.print_statistics()
    
    def print_statistics(self):
        """Print dataset statistics"""
        print("\n" + "="*50)
        print("DATASET STATISTICS")
        print("="*50)
        print(f"Total CAD Models: {self.manifest['statistics']['total_cad_models']}")
        print(f"Rendered Images: {self.manifest['statistics']['total_rendered_images']}")
        print(f"Augmented Images: {self.manifest['statistics']['total_augmented_images']}")
        print("\nCategories:")
        for category, parts in self.manifest["categories"].items():
            print(f"  {category}: {len(parts)} parts")
        print("="*50)


# ============================================================================
# PART 2: DOWNLOAD TRACKER (For Manual Collection)
# ============================================================================

class DownloadTracker:
    """Track manual downloads from McMaster/Grainger"""
    
    def __init__(self, manager):
        self.manager = manager
        self.checklist_file = manager.logs_dir / "download_checklist.json"
        self.load_checklist()
    
    def load_checklist(self):
        """Load download checklist"""
        if self.checklist_file.exists():
            with open(self.checklist_file, 'r') as f:
                self.checklist = json.load(f)
        else:
            self.checklist = {"pending": [], "completed": []}
    
    def save_checklist(self):
        """Save checklist"""
        with open(self.checklist_file, 'w') as f:
            json.dump(self.checklist, f, indent=2)
    
    def add_to_checklist(self, part_name, url, category):
        """Add part to download list"""
        item = {
            "name": part_name,
            "url": url,
            "category": category,
            "added": datetime.now().isoformat()
        }
        self.checklist["pending"].append(item)
        self.save_checklist()
        print(f"✓ Added to checklist: {part_name}")
    
    def mark_downloaded(self, part_name):
        """Mark part as downloaded"""
        for i, item in enumerate(self.checklist["pending"]):
            if item["name"] == part_name:
                item["downloaded"] = datetime.now().isoformat()
                self.checklist["completed"].append(item)
                del self.checklist["pending"][i]
                self.save_checklist()
                print(f"✓ Marked complete: {part_name}")
                return
        print(f"✗ Part not found: {part_name}")
    
    def show_pending(self):
        """Show pending downloads"""
        pending = self.checklist["pending"]
        
        if not pending:
            print("✓ All downloads complete!")
            return
        
        print(f"\n=== Pending Downloads ({len(pending)}) ===")
        for i, item in enumerate(pending, 1):
            print(f"{i}. {item['name']} ({item['category']})")
            print(f"   URL: {item['url']}")
        print()


# ============================================================================
# PART 3: BLENDER RENDERING SCRIPT
# ============================================================================

def create_blender_render_script(output_script_path="render_parts.py"):
    """
    Create a standalone Blender Python script for rendering
    Run this with: blender --background --python render_parts.py
    """
    
    blender_script = '''
import bpy
import math
import os
import sys
from pathlib import Path

def setup_scene():
    """Setup basic scene with lighting and camera"""
    # Delete default objects
    bpy.ops.object.select_all(action='SELECT')
    bpy.ops.object.delete()
    
    # Add camera
    bpy.ops.object.camera_add(location=(5, -5, 4))
    camera = bpy.context.object
    camera.rotation_euler = (math.radians(60), 0, math.radians(45))
    bpy.context.scene.camera = camera
    
    # Add sun light
    bpy.ops.object.light_add(type='SUN', location=(10, -10, 10))
    sun = bpy.context.object
    sun.data.energy = 3.0
    
    # Add fill light
    bpy.ops.object.light_add(type='AREA', location=(-5, 5, 5))
    fill = bpy.context.object
    fill.data.energy = 1.5
    
    # Set render settings
    bpy.context.scene.render.engine = 'CYCLES'
    bpy.context.scene.cycles.samples = 64
    bpy.context.scene.render.resolution_x = 512
    bpy.context.scene.render.resolution_y = 512
    bpy.context.scene.render.image_settings.file_format = 'JPEG'
    bpy.context.scene.render.film_transparent = False
    
    # Set white background
    bpy.context.scene.world.use_nodes = True
    bg = bpy.context.scene.world.node_tree.nodes['Background']
    bg.inputs[0].default_value = (1, 1, 1, 1)  # White

def import_cad_file(filepath):
    """Import CAD file (STEP, STL, OBJ)"""
    ext = filepath.suffix.lower()
    
    if ext in ['.step', '.stp']:
        # Import STEP
        try:
            bpy.ops.import_scene.step(filepath=str(filepath))
        except:
            print(f"Error: STEP import failed. Install STEP import add-on.")
            return None
    elif ext == '.stl':
        bpy.ops.import_mesh.stl(filepath=str(filepath))
    elif ext == '.obj':
        bpy.ops.import_scene.obj(filepath=str(filepath))
    elif ext in ['.iges', '.igs']:
        try:
            bpy.ops.import_scene.iges(filepath=str(filepath))
        except:
            print(f"Error: IGES import failed. Install IGES import add-on.")
            return None
    else:
        print(f"Unsupported file type: {ext}")
        return None
    
    # Get imported object
    obj = bpy.context.selected_objects[0] if bpy.context.selected_objects else None
    return obj

def center_and_scale_object(obj):
    """Center object at origin and scale to fit in view"""
    if not obj:
        return
    
    # Center object
    bpy.ops.object.select_all(action='DESELECT')
    obj.select_set(True)
    bpy.context.view_layer.objects.active = obj
    
    # Get bounding box
    bbox_corners = [obj.matrix_world @ Vector(corner) for corner in obj.bound_box]
    bbox_center = sum(bbox_corners, Vector()) / 8
    
    # Move to origin
    obj.location = -bbox_center
    
    # Scale to reasonable size
    max_dimension = max([max(abs(v[i]) for v in bbox_corners) for i in range(3)])
    if max_dimension > 0:
        scale_factor = 2.0 / max_dimension
        obj.scale = (scale_factor, scale_factor, scale_factor)

def render_part_multiple_views(cad_file, output_dir, num_views=24):
    """
    Render a part from multiple viewing angles
    
    Args:
        cad_file: Path to CAD file
        output_dir: Output directory for images
        num_views: Number of viewing angles (default 24 = 15° increments)
    """
    cad_path = Path(cad_file)
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    part_name = cad_path.stem
    
    print(f"\\nRendering: {part_name}")
    print(f"  File: {cad_file}")
    print(f"  Views: {num_views}")
    
    # Setup scene
    setup_scene()
    
    # Import CAD model
    obj = import_cad_file(cad_path)
    if not obj:
        print(f"  ✗ Failed to import {cad_file}")
        return []
    
    # Center and scale
    center_and_scale_object(obj)
    
    # Render from multiple angles
    rendered_files = []
    angle_step = 360 / num_views
    
    for i in range(num_views):
        # Rotate object around Z-axis
        angle = angle_step * i
        obj.rotation_euler[2] = math.radians(angle)
        
        # Set output path
        output_file = output_path / f"{part_name}_view_{i:03d}.jpg"
        bpy.context.scene.render.filepath = str(output_file)
        
        # Render
        bpy.ops.render.render(write_still=True)
        rendered_files.append(str(output_file))
        
        print(f"  Rendered view {i+1}/{num_views} ({angle:.0f}°)")
    
    # Clean up
    bpy.ops.object.select_all(action='SELECT')
    bpy.ops.object.delete()
    
    print(f"  ✓ Complete: {len(rendered_files)} images")
    return rendered_files

def batch_render_directory(cad_dir, output_dir, num_views=24):
    """Render all CAD files in a directory"""
    cad_path = Path(cad_dir)
    
    if not cad_path.exists():
        print(f"Error: Directory not found: {cad_dir}")
        return
    
    # Find all CAD files
    extensions = ['.step', '.stp', '.stl', '.obj', '.iges', '.igs']
    cad_files = []
    for ext in extensions:
        cad_files.extend(cad_path.rglob(f"*{ext}"))
    
    print(f"\\n{'='*60}")
    print(f"BATCH RENDERING")
    print(f"{'='*60}")
    print(f"CAD Directory: {cad_dir}")
    print(f"Output Directory: {output_dir}")
    print(f"Found {len(cad_files)} CAD files")
    print(f"Views per part: {num_views}")
    print(f"Total images to generate: {len(cad_files) * num_views}")
    print(f"{'='*60}\\n")
    
    all_rendered = []
    
    for idx, cad_file in enumerate(cad_files, 1):
        print(f"\\n[{idx}/{len(cad_files)}]")
        
        # Determine output subdirectory based on category
        relative_path = cad_file.relative_to(cad_path)
        category = relative_path.parts[0] if len(relative_path.parts) > 1 else "uncategorized"
        category_output = Path(output_dir) / category
        
        rendered = render_part_multiple_views(cad_file, category_output, num_views)
        all_rendered.extend(rendered)
    
    print(f"\\n{'='*60}")
    print(f"✓ RENDERING COMPLETE")
    print(f"  Total images generated: {len(all_rendered)}")
    print(f"{'='*60}\\n")
    
    return all_rendered

if __name__ == "__main__":
    # Get arguments from command line
    if len(sys.argv) < 4:
        print("Usage: blender --background --python render_parts.py -- <cad_dir> <output_dir> [num_views]")
        sys.exit(1)
    
    # Arguments after '--' are passed to the script
    args = sys.argv[sys.argv.index("--") + 1:]
    
    cad_directory = args[0]
    output_directory = args[1]
    num_views = int(args[2]) if len(args) > 2 else 24
    
    # Import Vector after bpy is available
    from mathutils import Vector
    
    # Run batch rendering
    batch_render_directory(cad_directory, output_directory, num_views)
'''
    
    with open(output_script_path, 'w') as f:
        f.write(blender_script)
    
    print(f"✓ Created Blender render script: {output_script_path}")
    return output_script_path


# ============================================================================
# PART 4: IMAGE AUGMENTATION PIPELINE
# ============================================================================

class ImageAugmentor:
    """Augment rendered images to increase dataset size"""
    
    def __init__(self, input_dir, output_dir):
        self.input_dir = Path(input_dir)
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
    
    def apply_motion_blur(self, img):
        """Apply motion blur"""
        kernel_size = np.random.randint(5, 25)
        angle = np.random.uniform(0, 180)
        
        kernel = np.zeros((kernel_size, kernel_size))
        kernel[kernel_size // 2, :] = np.ones(kernel_size)
        kernel = kernel / kernel_size
        
        center = (kernel_size // 2, kernel_size // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        kernel = cv2.warpAffine(kernel, M, (kernel_size, kernel_size))
        
        blurred = cv2.filter2D(img, -1, kernel)
        return blurred
    
    def random_crop(self, img, min_ratio=0.6, max_ratio=0.95):
        """Random crop and resize"""
        h, w = img.shape[:2]
        crop_ratio = np.random.uniform(min_ratio, max_ratio)
        
        crop_h = int(h * crop_ratio)
        crop_w = int(w * crop_ratio)
        
        start_h = np.random.randint(0, h - crop_h + 1)
        start_w = np.random.randint(0, w - crop_w + 1)
        
        cropped = img[start_h:start_h + crop_h, start_w:start_w + crop_w]
        resized = cv2.resize(cropped, (w, h), interpolation=cv2.INTER_LINEAR)
        
        return resized
    
    def scale_zoom(self, img, scale_range=(0.5, 1.5)):
        """Scale/zoom image"""
        h, w = img.shape[:2]
        scale_factor = np.random.uniform(scale_range[0], scale_range[1])
        
        new_h = int(h * scale_factor)
        new_w = int(w * scale_factor)
        
        scaled = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
        canvas = np.ones((h, w, 3), dtype=np.uint8) * 255
        
        if scale_factor > 1.0:
            start_h = (new_h - h) // 2
            start_w = (new_w - w) // 2
            canvas = scaled[start_h:start_h + h, start_w:start_w + w]
        else:
            start_h = (h - new_h) // 2
            start_w = (w - new_w) // 2
            canvas[start_h:start_h + new_h, start_w:start_w + new_w] = scaled
        
        return canvas
    
    def adjust_contrast(self, img_path, factor):
        """Adjust image contrast"""
        pil_img = Image.open(img_path)
        enhancer = ImageEnhance.Contrast(pil_img)
        contrasted = enhancer.enhance(factor)
        
        # Convert back to OpenCV format
        return cv2.cvtColor(np.array(contrasted), cv2.COLOR_RGB2BGR)
    
    def augment_image(self, image_path, num_augmentations=20):
        """
        Apply all augmentations to a single image
        
        Args:
            image_path: Path to input image
            num_augmentations: Total number of augmented versions to create
        """
        img = cv2.imread(str(image_path))
        if img is None:
            print(f"✗ Failed to load: {image_path}")
            return []
        
        base_name = Path(image_path).stem
        
        # Determine category from path
        relative_path = Path(image_path).relative_to(self.input_dir)
        category = relative_path.parts[0] if len(relative_path.parts) > 1 else "uncategorized"
        category_output = self.output_dir / category
        category_output.mkdir(parents=True, exist_ok=True)
        
        augmented_files = []
        per_technique = num_augmentations // 4
        
        # 1. Motion Blur
        for i in range(per_technique):
            blurred = self.apply_motion_blur(img.copy())
            output_path = category_output / f"{base_name}_motion_{i}.jpg"
            cv2.imwrite(str(output_path), blurred)
            augmented_files.append(str(output_path))
        
        # 2. Random Cropping
        for i in range(per_technique):
            cropped = self.random_crop(img.copy())
            output_path = category_output / f"{base_name}_crop_{i}.jpg"
            cv2.imwrite(str(output_path), cropped)
            augmented_files.append(str(output_path))
        
        # 3. Scaling/Zooming
        for i in range(per_technique):
            scaled = self.scale_zoom(img.copy())
            output_path = category_output / f"{base_name}_scale_{i}.jpg"
            cv2.imwrite(str(output_path), scaled)
            augmented_files.append(str(output_path))
        
        # 4. Contrast Adjustment
        for i in range(per_technique):
            factor = np.random.uniform(0.5, 2.0)
            contrasted = self.adjust_contrast(image_path, factor)
            output_path = category_output / f"{base_name}_contrast_{i}.jpg"
            cv2.imwrite(str(output_path), contrasted)
            augmented_files.append(str(output_path))
        
        return augmented_files
    
    def batch_augment(self, num_augmentations=20):
        """Augment all images in input directory"""
        
        # Find all images
        image_files = list(self.input_dir.rglob("*.jpg")) + \
                     list(self.input_dir.rglob("*.png")) + \
                     list(self.input_dir.rglob("*.jpeg"))
        
        print(f"\n{'='*60}")
        print(f"IMAGE AUGMENTATION")
        print(f"{'='*60}")
        print(f"Input Directory: {self.input_dir}")
        print(f"Output Directory: {self.output_dir}")
        print(f"Found {len(image_files)} images")
        print(f"Augmentations per image: {num_augmentations}")
        print(f"Total augmented images: {len(image_files) * num_augmentations}")
        print(f"{'='*60}\n")
        
        all_augmented = []
        
        for idx, img_path in enumerate(image_files, 1):
            print(f"[{idx}/{len(image_files)}] Augmenting: {img_path.name}")
            
            augmented = self.augment_image(img_path, num_augmentations)
            all_augmented.extend(augmented)
            
            if idx % 10 == 0:
                print(f"  Progress: {idx}/{len(image_files)} images processed")
        
        print(f"\n{'='*60}")
        print(f"✓ AUGMENTATION COMPLETE")
        print(f"  Total augmented images: {len(all_augmented)}")
        print(f"{'='*60}\n")
        
        return all_augmented


# ============================================================================
# PART 5: MAIN PIPELINE ORCHESTRATOR
# ============================================================================

class PartValidationPipeline:
    """Complete pipeline from CAD models to training dataset"""
    
    def __init__(self, project_name="part_validation_project"):
        self.manager = PartDatasetManager(project_name)
        self.tracker = DownloadTracker(self.manager)
        print(f"\n✓ Initialized project: {project_name}")
        self.manager.print_statistics()
    
    def step1_organize_cad_files(self, source_directory):
        """Step 1: Organize downloaded CAD files"""
        print("\n" + "="*60)
        print("STEP 1: ORGANIZE CAD FILES")
        print("="*60)
        self.manager.organize_cad_files(source_directory)
    
    def step2_render_images(self, num_views=24):
        """Step 2: Render CAD models to images using Blender"""
        print("\n" + "="*60)
        print("STEP 2: RENDER 3D MODELS")
        print("="*60)
        
        # Create Blender script
        script_path = self.manager.root / "render_parts.py"
        create_blender_render_script(script_path)
        
        # Instructions for running Blender
        cad_dir = self.manager.cad_dir
        render_dir = self.manager.rendered_dir
        
        command = f'blender --background --python "{script_path}" -- "{cad_dir}" "{render_dir}" {num_views}'
        
        print("\nTo render your CAD models, run this command:")
        print("\n" + "-"*60)
        print(command)
        print("-"*60)
        print("\nNote: This requires Blender to be installed and in your PATH")
        print("Download Blender from: https://www.blender.org/download/")
        
        # Save command to file
        command_file = self.manager.logs_dir / "render_command.txt"
        with open(command_file, 'w') as f:
            f.write(command)
        print(f"\nCommand saved to: {command_file}")
        
        return command
    
    def step3_augment_images(self, num_augmentations=20):
        """Step 3: Augment rendered images"""
        print("\n" + "="*60)
        print("STEP 3: AUGMENT IMAGES")
        print("="*60)
        
        augmentor = ImageAugmentor(
            self.manager.rendered_dir,
            self.manager.augmented_dir
        )
        
        augmented_files = augmentor.batch_augment(num_augmentations)
        
        # Update manifest
        self.manager.manifest["statistics"]["total_augmented_images"] = len(augmented_files)
        self.manager.save_manifest()
        
        return augmented_files
    
    def step4_create_training_splits(self, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15):
        """Step 4: Split dataset into train/val/test"""
        print("\n" + "="*60)
        print("STEP 4: CREATE TRAINING SPLITS")
        print("="*60)
        
        # Create split directories
        splits_dir = self.manager.root / "dataset_splits"
        train_dir = splits_dir / "train"
        val_dir = splits_dir / "val"
        test_dir = splits_dir / "test"
        
        for d in [train_dir, val_dir, test_dir]:
            d.mkdir(parents=True, exist_ok=True)
        
        # Get all augmented images by category
        for category in self.manager.manifest["categories"].keys():
            category_path = self.manager.augmented_dir / category
            
            if not category_path.exists():
                continue
            
            images = list(category_path.glob("*.jpg"))
            np.random.shuffle(images)
            
            n_train = int(len(images) * train_ratio)
            n_val = int(len(images) * val_ratio)
            
            train_images = images[:n_train]
            val_images = images[n_train:n_train + n_val]
            test_images = images[n_train + n_val:]
            
            # Create category folders in each split
            for split_dir in [train_dir, val_dir, test_dir]:
                (split_dir / category).mkdir(exist_ok=True)
            
            # Copy images
            for img in train_images:
                shutil.copy2(img, train_dir / category / img.name)
            
            for img in val_images:
                shutil.copy2(img, val_dir / category / img.name)
            
            for img in test_images:
                shutil.copy2(img, test_dir / category / img.name)
            
            print(f"\n{category}:")
            print(f"  Train: {len(train_images)}")
            print(f"  Val: {len(val_images)}")
            print(f"  Test: {len(test_images)}")
        
        print(f"\n✓ Dataset splits created in: {splits_dir}")
        
        return splits_dir
    
    def run_full_pipeline(self, source_cad_dir, num_views=24, num_augmentations=20):
        """Run complete pipeline"""
        print("\n" + "="*70)
        print(" "*15 + "PART VALIDATION DATASET PIPELINE")
        print("="*70)
        
        # Step 1: Organize
        self.step1_organize_cad_files(source_cad_dir)
        
        # Step 2: Render (generates command)
        render_command = self.step2_render_images(num_views)
        
        print("\n" + "="*70)
        print("PIPELINE PAUSED - Manual Step Required")
        print("="*70)
        print("\nPlease run the Blender rendering command shown above.")
        print("After rendering is complete, run:")
        print("  pipeline.step3_augment_images()")
        print("  pipeline.step4_create_training_splits()")
        print("="*70)
        
        return render_command
    
    def generate_dataset_report(self):
        """Generate final dataset report"""
        report_path = self.manager.logs_dir / "dataset_report.txt"
        
        with open(report_path, 'w') as f:
            f.write("="*70 + "\n")
            f.write("PART VALIDATION DATASET REPORT\n")
            f.write("="*70 + "\n\n")
            
            f.write(f"Project: {self.manager.root}\n")
            f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
            
            f.write("STATISTICS\n")
            f.write("-"*70 + "\n")
            stats = self.manager.manifest["statistics"]
            f.write(f"Total CAD Models: {stats['total_cad_models']}\n")
            f.write(f"Total Rendered Images: {stats['total_rendered_images']}\n")
            f.write(f"Total Augmented Images: {stats['total_augmented_images']}\n\n")
            
            f.write("CATEGORIES\n")
            f.write("-"*70 + "\n")
            for category, parts in self.manager.manifest["categories"].items():
                f.write(f"{category}: {len(parts)} parts\n")
            
            f.write("\n" + "="*70 + "\n")
        
        print(f"\n✓ Report generated: {report_path}")
        
        with open(report_path, 'r') as f:
            print(f.read())


# ============================================================================
# PART 6: COMMAND LINE INTERFACE
# ============================================================================

def main():
    """Main entry point with interactive menu"""
    
    print("\n" + "="*70)
    print(" "*15 + "2.156 PART VALIDATION DATASET BUILDER")
    print("="*70)
    
    project_name = input("\nEnter project name (default: part_validation_project): ").strip()
    if not project_name:
        project_name = "part_validation_project"
    
    pipeline = PartValidationPipeline(project_name)
    
    while True:
        print("\n" + "="*70)
        print("MENU")
        print("="*70)
        print("1. Organize CAD files from downloads folder")
        print("2. Generate Blender rendering command")
        print("3. Augment rendered images")
        print("4. Create train/val/test splits")
        print("5. Run full pipeline (steps 1-2)")
        print("6. Generate dataset report")
        print("7. View download checklist")
        print("8. Add part to download checklist")
        print("9. Mark part as downloaded")
        print("0. Exit")
        print("="*70)
        
        choice = input("\nSelect option: ").strip()
        
        if choice == "1":
            source_dir = input("Enter source directory with CAD files: ").strip()
            pipeline.step1_organize_cad_files(source_dir)
        
        elif choice == "2":
            num_views = input("Number of views per part (default 24): ").strip()
            num_views = int(num_views) if num_views else 24
            pipeline.step2_render_images(num_views)
        
        elif choice == "3":
            num_aug = input("Augmentations per image (default 20): ").strip()
            num_aug = int(num_aug) if num_aug else 20
            pipeline.step3_augment_images(num_aug)
        
        elif choice == "4":
            pipeline.step4_create_training_splits()
        
        elif choice == "5":
            source_dir = input("Enter source directory with CAD files: ").strip()
            num_views = input("Number of views per part (default 24): ").strip()
            num_views = int(num_views) if num_views else 24
            pipeline.run_full_pipeline(source_dir, num_views)
        
        elif choice == "6":
            pipeline.generate_dataset_report()
        
        elif choice == "7":
            pipeline.tracker.show_pending()
        
        elif choice == "8":
            name = input("Part name: ").strip()
            url = input("URL: ").strip()
            category = input("Category: ").strip()
            pipeline.tracker.add_to_checklist(name, url, category)
        
        elif choice == "9":
            name = input("Part name: ").strip()
            pipeline.tracker.mark_downloaded(name)
        
        elif choice == "0":
            print("\nGoodbye!")
            break
        
        else:
            print("\n✗ Invalid option")


# ============================================================================
# QUICK START EXAMPLE
# ============================================================================

if __name__ == "__main__":
    # Uncomment one of these modes:
    
    # MODE 1: Interactive menu
    main()
    
    # MODE 2: Quick automated run
    # pipeline = PartValidationPipeline("my_2156_project")
    # pipeline.run_full_pipeline("path/to/downloaded_cad_files", num_views=24, num_augmentations=20)
    
    # MODE 3: Step by step
    # pipeline = PartValidationPipeline("my_2156_project")
    # pipeline.step1_organize_cad_files("downloads")
    # # ... run blender rendering manually ...
    # pipeline.step3_augment_images(20)
    # pipeline.step4_create_training_splits()
    # pipeline.generate_dataset_report()